In [39]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import inspect
import os
sns.set(style="white")

In [41]:
FIG_PATH = "./figure/"
espl = pd.read_pickle("espl.pkl")
y = pd.read_pickle("./y-2s.pkl")

In [42]:
espl.head()

,intTrapz,maxA,MVDeriv,meanA,Var,Med
0,14398.936269,17.920838,0.001170,9.661280,13.439433,8.559716
1,15477.238332,19.667330,0.001723,10.383013,15.650140,9.920799
2,15260.431960,19.205382,0.001222,10.253074,16.677828,9.632026
3,14983.215855,19.026058,0.001365,10.073228,14.020140,9.310040
4,15871.838644,17.863954,0.001457,10.640028,16.545959,10.268722


In [43]:
from funzioni import indice_gini, tasso_errata_classificazione, MatriceConfusione

In [44]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.metrics import confusion_matrix
import graphviz

In [45]:
X_train, X_val, y_train, y_val = train_test_split(espl, y, test_size=0.25, random_state=42)

In [46]:
dtc = Pipeline([ 
    ("tree", DecisionTreeClassifier(random_state=42,criterion='entropy'))
])

dtc.fit(X_train, y_train)

#y_pred = dtc.predict(X_val)
#dtc_acc = accuracy_score(y_val, y_pred)

#print('Accuratezza DummyClassifier("most_frequent"): {:.1f}%'.format(100 * dc_acc))
#print("Accuratezza DecisionTreeClassifier(): {:.2f}%".format(100 * dtc_acc))
#MatriceConfusione(y_val, y_pred)
#plt.show()

Pipeline(memory=None,
     steps=[('tree', DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best'))])

Pipeline(memory=None,
     steps=[('tree', DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best'))])

In [ ]:
dot_data = export_graphviz(
    decision_tree=dtc.named_steps["tree"], 
    feature_names=espl.columns,
    
    filled=True,
    rounded=True,
    node_ids=None
)
display(graphviz.Source(dot_data))

In [48]:
print("Profondità dell'albero allenato senza restrizioni: {}".format(dtc.named_steps["tree"].tree_.max_depth))
print("Massimo numero minimo di osservazioni in una foglia: {}".format(len(X_train) // 2))

Profondità dell'albero allenato senza restrizioni: 12
Massimo numero minimo di osservazioni in una foglia: 873


In [49]:
from sklearn.model_selection import ParameterGrid
import tqdm

In [51]:
param_grid = ParameterGrid({
    'tree__max_depth': np.arange(1, dtc.named_steps["tree"].tree_.max_depth),
    'tree__min_samples_leaf': 2 ** np.arange(10),
})
print(param_grid.param_grid)

[{'tree__max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11]), 'tree__min_samples_leaf': array([  1,   2,   4,   8,  16,  32,  64, 128, 256, 512], dtype=int32)}]


In [61]:
len(X_train)

1746

In [57]:
len(dtc.named_steps["tree"].apply(X_train, check_input=True))

1746

In [63]:
import numpy
numpy.eye(3)

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [ ]:
risultati = []

for params in tqdm.tqdm(param_grid):
    dtc.set_params(**params)
    dtc.fit(X_train, y_train)
    y_pred = dtc.predict(X_val)
    params["entropy"] = accuracy_score(y_val, y_pred)
    risultati.append(params)

risultati = pd.DataFrame(risultati).sort_values(["accuracy_score", "tree__max_depth"], ascending=[False, True])
risultati.reset_index(drop=True, inplace=True)
print("Primi 5:")
display(risultati.head())

print("Ultimi 5:")
risultati.tail()